# TODO: Add CNN Model

# TODO Try with ray tune

In [1]:
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

import pygame
import numpy as np

import torch
from IPython.display import clear_output
from IPython import display
import random
import matplotlib.pylab as plt
import copy
import time

from typing import List, Optional
from gym.envs.toy_text.frozen_lake import generate_random_map

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class WrappedFrozenLake(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.old = 0
        self.desc = self.env.desc
        self.size = len(self.desc)
        self.observation_space = gym.spaces.Box(0,1,shape=(3,self.size,self.size,),dtype="float32")
        self.action_space = self.env.action_space
        
    def oneHot(self,s):
        x = np.zeros(self.size*self.size)
        x[s] = 1
        state_ = np.array([ x.reshape(self.size,self.size),
                         np.array(self.env.desc == b"F").astype("float32"),
                         np.array(self.env.desc == b"G").astype("float32")
                          ])
        return state_.reshape(3,self.size,self.size,)

    def reset(self):
        # return self.oneHot(1)
        return self.oneHot(self.env.reset())
    
    def step(self, action):
        obs, r, done, info = self.env.step(action)
        
        if done:
            reward = 2 if r > 0 else -1
        elif obs == self.old:
            reward = -1
        else:
            reward = 0
        self.old = obs
        return self.oneHot(obs), reward, done, info

from ray.tune.registry import register_env

def env_creator(env_config): 
    size = env_config['size']
    numHoles = env_config['numHoles']
    p = 1-numHoles/(size**2)
    desc = generate_random_map(size=size, p=p)
    return WrappedFrozenLake(gym.make('FrozenLake-v1', desc = desc, is_slippery=False))  # return an env instance

register_env("myenv", env_creator)

In [3]:
env = env_creator(env_config = {'size':6, 'numHoles': 6})
env.reset()
done = False

while not done:
    # plt.cla()
    display.clear_output(wait=True)

    obs, reward, done, info = env.step(env.action_space.sample())
    env.render()
    time.sleep(0.1)
    # display.clear_output(wait=True)
    # display.display(plt.gcf())
    # plt.gcf()

  (Down)
SFFFFF
FFFFFF
FFFFFF
FFFFFF
FHFFFH
FFFFFG


In [4]:
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.framework import try_import_tf, try_import_torch

torch, nn = try_import_torch()

class MyTorchModel(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        """Build a simple [16, 16]-MLP (+ value branch)."""
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)
        self.device = torch.device("cpu")#"cuda"
        #                            if torch.cuda.is_available() else "cpu")

        self.mainLayer = nn.Sequential(
                # nn.Conv2d(3,12,kernel_size=3,stride= 1,padding=1),
                # nn.ReLU(),
                # nn.Conv2d(12,24,kernel_size=3,stride=1,padding=1),
                # nn.ReLU(),
                # nn.Conv2d(24,36,kernel_size=3,stride=1,padding=1),
                # nn.ReLU(),            
                nn.Flatten(start_dim=-3,end_dim=-1),
                nn.Linear(108,256),
                nn.ReLU(),
                nn.Linear(256,256),
                nn.ReLU(),
        ).to(self.device)
        
        # Action logits output.
        self.layer_out = nn.Linear(256, num_outputs).to(self.device)

        # "Value"-branch (single node output).
        # Used by several RLlib algorithms (e.g. PPO) to calculate an observation's value.
        self.value_branch = nn.Linear(256, 1).to(self.device)
        self.cur_value = None

    def forward(self, input_dict, state, seq_lens):
        """Custom-define your forard pass logic here."""
        # Pass inputs through our 2 layers.
        layer_1_out = self.mainLayer(input_dict["obs"])
        logits = self.layer_out(layer_1_out)

        # Calculate the "value" of the observation and store it for
        # when `value_function` is called.
        self.cur_value = self.value_branch(layer_1_out).squeeze(-1)

        return logits, state

    def value_function(self):
        """Implement the value branch forward pass logic here:
        
        We will just return the already calculated `self.cur_value`.
        """
        assert self.cur_value is not None, "Must call `forward()` first!"
        return self.cur_value

In [5]:
size = 6
test_model_torch = MyTorchModel(
   obs_space=gym.spaces.Box(0,1,shape=(3,size,size,), dtype=np.int32),
   action_space=gym.spaces.Discrete(4),
   num_outputs=4,
   model_config={},
   name="MyModel",
)
#print("Torch-output={}".format(test_model_torch({"obs": torch.from_numpy(np.array([[0.5, 0.5]], dtype=np.float32))})))

obs = gym.spaces.Box(0,1,shape=(3,size,size)).sample()
test_model_torch({"obs": torch.from_numpy(obs)})

(tensor([-0.0119, -0.0290,  0.1312,  0.0470], grad_fn=<AddBackward0>), [])

In [6]:
config = {
        "framework": "torch",
        "env":"myenv",  
        "env_config":{'size':6, 'numHoles': 6},
        "num_workers": 6,
          "model": {
            # "fcnet_hiddens": [512, 1024, 256],
            # "fcnet_activation": "relu",
             "custom_model": MyTorchModel,  # for torch users: "custom_model": MyTorchModel
             "custom_model_config": {},
          },
#        "num_workers": 1,
        'num_envs_per_worker': 1,
        # 'gamma': 1.0,
        # 'lr': 0.001,
        # 'train_batch_size': 1000,  
        "create_env_on_driver": True,
}

trainer = DQNTrainer(config= config)

In [7]:
policy = trainer.get_policy()

In [9]:
keepTraining = True
maxIter = 50
i = 0

while keepTraining and i < (maxIter):  
    i += 1 
    results = trainer.train()
    if results['episode_reward_mean'] > 1.99:
        keepTraining = False
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")

Iter: 1; avg. reward=1.4
Iter: 2; avg. reward=1.32
Iter: 3; avg. reward=1.05
Iter: 4; avg. reward=0.7
Iter: 5; avg. reward=0.74
Iter: 6; avg. reward=0.5
Iter: 7; avg. reward=0.62
Iter: 8; avg. reward=0.91
Iter: 9; avg. reward=1.29
Iter: 10; avg. reward=1.23
Iter: 11; avg. reward=1.17
Iter: 12; avg. reward=1.22
Iter: 13; avg. reward=1.62
Iter: 14; avg. reward=1.76
Iter: 15; avg. reward=1.81
Iter: 16; avg. reward=1.89
Iter: 17; avg. reward=1.69
Iter: 18; avg. reward=1.55
Iter: 19; avg. reward=1.39
Iter: 20; avg. reward=1.57
Iter: 21; avg. reward=1.5
Iter: 22; avg. reward=1.58
Iter: 23; avg. reward=1.63
Iter: 24; avg. reward=1.51
Iter: 25; avg. reward=1.4
Iter: 26; avg. reward=1.24
Iter: 27; avg. reward=1.38
Iter: 28; avg. reward=1.6
Iter: 29; avg. reward=1.62
Iter: 30; avg. reward=1.45
Iter: 31; avg. reward=1.29
Iter: 32; avg. reward=1.43
Iter: 33; avg. reward=0.79
Iter: 34; avg. reward=0.79
Iter: 35; avg. reward=0.71
Iter: 36; avg. reward=1.52
Iter: 37; avg. reward=1.58
Iter: 38; avg. r

In [10]:
# We use the `Trainer.save()` method to create a checkpoint.
checkpoint_file = trainer.save()
print(f"Trainer (at iteration {trainer.iteration} was saved in '{checkpoint_file}'!")

# Here is what a checkpoint directory contains:
print("The checkpoint directory contains the following files:")
import os
os.listdir(os.path.dirname(checkpoint_file))

Trainer (at iteration 100 was saved in '/home/ajit.kumar@SNU.IN/ray_results/DQNTrainer_myenv_2022-07-16_13-11-35ybmiq79b/checkpoint_000100/checkpoint-100'!
The checkpoint directory contains the following files:


['checkpoint-100.tune_metadata', 'checkpoint-100', '.is_checkpoint']

In [12]:
eval_config = config.copy()
#eval_config['explore'] = False
eval_trainer2 = PPOTrainer(config=eval_config)
eval_trainer2.restore(checkpoint_file)

results = eval_trainer2.evaluate()
print(f"Evaluating new trainer: R={results['evaluation']['episode_reward_mean']}")

2022-07-16 13:18:08,604	WARNING ppo.py:386 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=6 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 666.


ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

(scheduler +7m27s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +7m27s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


In [13]:
results = eval_trainer2.evaluate()
print(f"Evaluating new trainer: R={results['evaluation']['episode_reward_mean']}")

Evaluating new trainer: R=-2.2215700660308144


In [7]:
trainer.load_checkpoint('model/checkpoint_000050/checkpoint-50')

In [25]:
won = 0
lost = 0
rewardList = []

for i in range(100):
    env = env_creator({'size':6, 'numHoles': 6}) #gym.make("myenv-v1")#"FrozenLake-v1")
    #, #FrozenLakeWrapped({})
    # Get the initial observation (should be: [0.0] for the starting position).
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    while not done:
        # plt.cla()
#        display.clear_output(wait=True)
        # print("num won: ", won, " played: ", i, "total reward: ", total_reward)
        # env.render()

        action = new_trainer.compute_single_action(obs,explore=False)
        obs, reward, done, info = env.step(action)
        #print(obs, reward, done)
        total_reward += reward
        # time.sleep(0.1)
        
        # display.display(plt.gcf())
        # plt.gcf()
        if done: 
            rewardList.append(total_reward)
    if reward > 0 :
        won +=1 
      
    
#print(rewardList, np.mean(rewardList))
print( np.mean(rewardList))


-0.13


In [26]:
won

29

In [14]:
trainer.compute_single_action(obs)

2

In [32]:
env = env_creator({'size':6, 'numHoles': 6})# gym.make("FrozenLake-v1")
obs = env.reset()
done = False
total_reward = 0.0
# Play one episode.
while not done:
    display.clear_output(wait=True)
    obs, reward, done, _ = env.step(new_trainer.compute_single_action(obs,explore=False))
    env.render()
    time.sleep(0.1)
    if done: 
        rewardList.append(total_reward)
if reward > 0 :
    won +=1 


  (Down)
SFFFFF
FFFHFF
FFHFFF
FFFFFF
FFFFHF
FFFHFG


In [29]:
trainer.stop()
ray.shutdown()

In [ ]:

from ray import tune

# Running stuff with tune, we can re-use the exact
# same config that we used when working with RLlib directly!
tune_config = config.copy()

# Let's add our first hyperparameter search via our config.
# How about we try two different learning rates? Let's say 0.00005 and 0.5 (ouch!).
tune_config["lr"] = tune.grid_search([0.0001])  # <- 0.5? again: ouch!
#tune_config["train_batch_size"] = tune.grid_search([4000])

# Now that we will run things "automatically" through tune, we have to
# define one or more stopping criteria.
# Tune will stop the run, once any single one of the criteria is matched (not all of them!).
stop = {
    # Note that the keys used here can be anything present in the above `rllib_trainer.train()` output dict.
    "training_iteration": 50,
    "episode_reward_mean": 1.95,
}

# "PPO" is a registered name that points to RLlib's PPOTrainer.
# See `ray/rllib/agents/registry.py`

# Run a simple experiment until one of the stopping criteria is met.
analysis =  tune.run(
    "PPO",
    config=tune_config,
    stop=stop,

    # Note that no trainers will be returned from this call here.
    # Tune will create n Trainers internally, run them in parallel and destroy them at the end.
    # However, you can ...
    checkpoint_at_end=True,  # ... create a checkpoint when done.
    checkpoint_freq=1,  # ... create a checkpoint every 10 training iterations.
)

In [8]:
# The previous tune.run (the one we did before the exercise) returned an Analysis object, from which we can access any checkpoint
# (given we set checkpoint_freq or checkpoint_at_end to reasonable values) like so:
print(analysis)
# Get all trials (we only have one).
trials = analysis.trials
# Assuming, the first trial was the best, we'd like to extract this trial's best checkpoint "":
best_checkpoint = analysis.get_best_checkpoint(trial=trials[0], metric="episode_reward_mean", mode="max")
print(f"Found best checkpoint for trial #2: {best_checkpoint}")


Found best checkpoint for trial #2: /home/ajit.kumar@SNU.IN/ray_results/PPO/PPO_myenv_4159a_00000_0_lr=0.0001_2022-07-16_12-46-48/checkpoint_000017/checkpoint-17


In [23]:
rllib_config = tune_config.copy()
rllib_config["lr"] = 0.0001
rllib_config["train_batch_size"] = 4159
#rllib_config["explore"] = False

# Restore a RLlib Trainer from the checkpoint.
new_trainer = PPOTrainer(config=rllib_config)
new_trainer.restore(best_checkpoint)

2022-07-16 12:59:09,211	WARNING ppo.py:386 -- `train_batch_size` (4159) cannot be achieved with your other settings (num_workers=6 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 693.
2022-07-16 12:59:09,329	INFO trainable.py:588 -- Restored on 10.13.62.8 from checkpoint: /home/ajit.kumar@SNU.IN/ray_results/PPO/PPO_myenv_4159a_00000_0_lr=0.0001_2022-07-16_12-46-48/checkpoint_000017/checkpoint-17
2022-07-16 12:59:09,330	INFO trainable.py:597 -- Current state after restoring: {'_iteration': 17, '_timesteps_total': None, '_time_total': 122.87536454200745, '_episodes_total': 16946}


In [ ]:
new_trainer.evaluate()

In [35]:
analysis.trials[0]

PPO_myenv_4159a_00000

In [36]:
x = [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]

In [37]:
len(x)

796